<a href="https://colab.research.google.com/github/SBoo9/TRANSFORMERS_AND_BERT/blob/main/BERT_TEST_MODULE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install tensorflow_text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text

In [2]:
preprocessor_url="https://kaggle.com/models/tensorflow/albert/TensorFlow2/en-preprocess/3"
encoder_url="https://www.kaggle.com/models/tensorflow/albert/TensorFlow2/en-base/3"

In [3]:
bert_preprocess_model=hub.KerasLayer(preprocessor_url)

In [4]:
text_text=['Ratatouie was nice movie','I love fears of fathom']
text_preprocessed=bert_preprocess_model(text_text)
text_preprocessed.keys()

dict_keys(['input_type_ids', 'input_mask', 'input_word_ids'])

In [5]:
text_preprocessed['input_mask']

<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>

In [6]:
bert_model=hub.KerasLayer(encoder_url)
bert_result=bert_model(text_preprocessed)
bert_result.keys()

dict_keys(['pooled_output', 'sequence_output', 'encoder_outputs', 'default'])

In [7]:
bert_result['pooled_output']

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[ 0.75923294, -0.80510086,  0.81026065, ...,  0.24747384,
        -0.99964064,  0.80486214],
       [ 0.6398371 , -0.6786815 ,  0.8167322 , ..., -0.02645584,
        -0.9992816 ,  0.6906628 ]], dtype=float32)>

In [8]:
##!pip install tensorflow_text

In [9]:
from transformers import BertModel,BertConfig

In [10]:
configuration=BertConfig()
configuration

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [11]:
model=BertModel(configuration)

In [12]:
configuration=model.config

In [13]:
configuration

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [14]:
from transformers import AutoTokenizer, TFBertModel
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertModel.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs.last_hidden_state

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [15]:
outputs

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 8, 768), dtype=float32, numpy=
array([[[-0.11437166,  0.1937139 ,  0.12495933, ..., -0.38269064,
          0.21065918,  0.54070777],
        [ 0.5308244 ,  0.32074884,  0.36645907, ..., -0.00360633,
          0.7578604 ,  0.03884321],
        [-0.48765135,  0.88492507,  0.42556414, ..., -0.69762045,
          0.44583377,  0.12309456],
        ...,
        [-0.7002792 , -0.18150613,  0.32969645, ..., -0.4837929 ,
          0.06802386,  0.8900848 ],
        [-1.0354635 , -0.2566772 , -0.03165276, ...,  0.31974328,
          0.39990258,  0.17954789],
        [ 0.60799193,  0.26097047, -0.3130718 , ...,  0.0310978 ,
         -0.6282718 , -0.19942468]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-7.19458222e-01, -2.14449570e-01, -2.95759737e-01,
         3.66029292e-01,  2.79677540e-01,  2.21835095e-02,
         5.72990894e-01,  6.23308346e-02,  5.95868453e-0

In [16]:
last_hidden_states

<tf.Tensor: shape=(1, 8, 768), dtype=float32, numpy=
array([[[-0.11437166,  0.1937139 ,  0.12495933, ..., -0.38269064,
          0.21065918,  0.54070777],
        [ 0.5308244 ,  0.32074884,  0.36645907, ..., -0.00360633,
          0.7578604 ,  0.03884321],
        [-0.48765135,  0.88492507,  0.42556414, ..., -0.69762045,
          0.44583377,  0.12309456],
        ...,
        [-0.7002792 , -0.18150613,  0.32969645, ..., -0.4837929 ,
          0.06802386,  0.8900848 ],
        [-1.0354635 , -0.2566772 , -0.03165276, ...,  0.31974328,
          0.39990258,  0.17954789],
        [ 0.60799193,  0.26097047, -0.3130718 , ...,  0.0310978 ,
         -0.6282718 , -0.19942468]]], dtype=float32)>

In [17]:
from transformers import AutoTokenizer, TFBertForMaskedLM
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="tf")
logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = tf.where((inputs.input_ids == tokenizer.mask_token_id)[0])
selected_logits = tf.gather_nd(logits[0], indices=mask_token_index)

predicted_token_id = tf.math.argmax(selected_logits, axis=-1)
tokenizer.decode(predicted_token_id)

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


'paris'

In [18]:
labels = tokenizer("The capital of France is Paris.", return_tensors="tf")["input_ids"]
# mask labels of non-[MASK] tokens
labels = tf.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(float(outputs.loss), 2)

0.88

In [19]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForNextSentencePrediction

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertForNextSentencePrediction.from_pretrained("google-bert/bert-base-uncased")

prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors="tf")

logits = model(encoding["input_ids"], token_type_ids=encoding["token_type_ids"])[0]
assert logits[0][0] < logits[0][1]  # the next sentence was random

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [20]:
from transformers import AutoTokenizer, TFBertForSequenceClassification
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("ydshieh/bert-base-uncased-yelp-polarity")
model = TFBertForSequenceClassification.from_pretrained("ydshieh/bert-base-uncased-yelp-polarity")

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")

logits = model(**inputs).logits

predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some layers from the model checkpoint at ydshieh/bert-base-uncased-yelp-polarity were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ydshieh/bert-base-uncased-yelp-polarity.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


'LABEL_1'

In [21]:
from transformers import pipeline
claddifier=pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [22]:
classifier=claddifier

In [23]:
classifier("Ur father was a dog, but a great one")

[{'label': 'POSITIVE', 'score': 0.9959536790847778}]

In [24]:
classifier=pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [25]:
print(classifier("जी नहीं, किरण दूध नहीं पीती है"))

[{'label': 'Very Negative', 'score': 0.5678442716598511}]


In [26]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [27]:
model_name="nlptown/bert-base-multilingual-uncased-sentiment"
model=TFAutoModelForSequenceClassification.from_pretrained(model_name,from_pt=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [28]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
classifier=pipeline('sentiment-analysis',model=model,tokenizer=tokenizer)

Device set to use 0


In [29]:
classifier("will someone see this ever")

[{'label': '5 stars', 'score': 0.6888329982757568}]

In [30]:
inputs=tokenizer("We are happy, on your absence")

In [31]:
print(inputs)

{'input_ids': [101, 11312, 10320, 19308, 117, 10125, 12787, 27621, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
